# IMPORTS

import pandas as pd
from os import walk, path
import os
import sys
%matplotlib inline

In [13]:
NUMERIC_DIR = r"C:\Users\Dana\Documents\Final_Project\Data\processed\numeric_files"
CATEGORICAL_DIR = r"C:\Users\Dana\Documents\Final_Project\Data\processed\categorial_files"
META_DIR = r"C:\Users\Dana\Documents\Final_Project\Data\processed\meta_files"
processed_FOLDER = r"C:\Users\Dana\Documents\Final_Project\Data\processed"
ORDINAL_DIR = r"C:\Users\Dana\Documents\Final_Project\Data\processed\ordinal_files"

# All Data

### Remove: first 10 seconds from monitored data + outliers

When connecting monitored device to a patient, the first 10 minutes of the connection result from the calibration of the device, so the values are irrelevant

In [3]:
monitor_list = ('ArtBPD.csv','ArtBPM.csv','ArtBPS.csv','ETCO2.csv','FiO2.csv','HR.csv','Pat. RR.csv','PEEP.csv','PiP-Peak.csv','Pmax.csv','RR.csv','SaturationPulse.csv','Temp.csv','TotalMinuteVolume.csv','VTsetting.csv')

In [4]:
known_ouliers_dict = {"HR.csv":(20,240), "ArtBPM.csv":(0,200), "ArtBPD.csv":(0,200), "ArtBPS.csv":(0,350), "Temp.csv":(25,45)}

In [122]:
def remove_first_ten_minutes(df):
    df["time"] = pd.to_datetime(df["time"])
    df = df.sort_values(["key","time"])
    drop_this = df.groupby("key").first()["time"].to_frame("first_time")
    drop_this["plus_10"] = drop_this["first_time"] + pd.Timedelta(minutes=10)
    df = df.merge(drop_this, left_on="key", how="left", right_index=True)
    df["time_diff"] = df["time"] - df["plus_10"]
    df = df[df["time_diff"] > pd.Timedelta(minutes=0)]
    return df


In [123]:
def remove_known_outliers(df,f):
    df = df[df["value"].between(known_ouliers_dict[f][0], known_ouliers_dict[f][1])]
    return df

In [124]:
def remove_outliers(df):
    highest_val = df["value"].mean() + 4 * df["value"].std()
    df = df[df["value"] <= highest_val ]
    return df

In [127]:
for root, dirs, files in os.walk(NUMERIC_DIR):
    for f in files:
        df_parameter = pd.read_csv(os.path.join(NUMERIC_DIR,f))
        if f in monitor_list:
            df_parameter = remove_first_ten_minutes(df_parameter)
        df_parameter = df_parameter[df_parameter["value"] > 0]
        if f in known_ouliers_dict:
            df_parameter = remove_known_outliers(df_parameter,f)
        else:
            df_parameter = remove_outliers(df_parameter)
        df_parameter.to_csv(os.path.join(NUMERIC_DIR,f))

RR.csv
SaturationPulse.csv
SuctionsPerDay.csv
Temp.csv
TotalMinuteVolume.csv
Urea.csv
VTsetting.csv
WCC.csv


# Meta Data 

### Preparing boolean values where needed

Some of the data has numeric ot string values, but we only want to know whether a patient did or did not have the paramter

In [5]:
parameters_list = ('acmv.csv','AntibioticsCont.csv','AntibioticsIV.csv','psv.csv','simv.csv','SolumedrolUse.csv')

In [ ]:
for root, dirs, files in os.walk(META_DIR):
    for f in files:
        if f in parameters_list:
            df_preparing_bool = pd.read_csv(os.path.join(META_DIR,f))
            df_preparing_bool['value'] = 1
            df_preparing_bool.to_csv(os.path.join(META_DIR,f))

### Fixing duplicates 

In [11]:
for root, dirs, files in os.walk(META_DIR):
    for f in files:
        df_with_duplicates = pd.read_csv(os.path.join(META_DIR,f))
        df_with_duplicates["time"] = pd.to_datetime(df_with_duplicates["time"])
        df_with_duplicates = df_with_duplicates.sort_values(["key","time"])
        df_remove_duplicates = df_with_duplicates.groupby("key").last()
        df_remove_duplicates.to_csv(os.path.join(META_DIR,f))

# Ordinal

In [31]:
GCS_SOFA = pd.read_csv(r"C:\Users\Dana\Documents\Final_Project\Data\processed\ordinal_files\GCS_SOFA.csv")
GCS_SOFA = GCS_SOFA.replace({"value":{'< 6':0, '6 - 9':1, '10 - 12':2, '13 - 14':3,' > 14':4}})
GCS_SOFA.to_csv(os.path.join(r"C:\Users\Dana\Documents\Final_Project\Data\processed\ordinal_files\GCS_SOFA.csv"))

In [32]:
PaO2_FiO2_SOFA = pd.read_csv(r"C:\Users\Dana\Documents\Final_Project\Data\processed\ordinal_files\PaO2_FiO2_SOFA.csv")
PaO2_FiO2_SOFA = PaO2_FiO2_SOFA.replace({"value":{'<100':0, '<200':1, '<300':2, '<400':3, '> 400':4}})
PaO2_FiO2_SOFA.to_csv(os.path.join(r"C:\Users\Dana\Documents\Final_Project\Data\processed\ordinal_files\PaO2_FiO2_SOFA.csv"))